In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '1'

In [2]:
import sys

SOURCE_DIR = os.path.dirname(os.path.dirname(os.path.abspath(__name__)))
sys.path.insert(0, SOURCE_DIR)

In [3]:
import malaya_speech
import malaya_speech.config
import malaya_speech.train.model.conformer as conformer
import malaya_speech.train.model.transducer as transducer
import tensorflow as tf






The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.




In [4]:
config = malaya_speech.config.transducer_featurizer_config
featurizer = malaya_speech.utils.tf_featurization.STTFeaturizer(**config)

In [5]:
config = malaya_speech.config.conformer_small_encoder_config
small_model = conformer.Model(**config)

In [6]:
small_model.conv_subsampling.time_reduction_factor

4

In [7]:
k = 0.0062169171196134855

i = tf.placeholder(tf.float32, [None, None])
lengths = tf.placeholder(tf.float32, [None])
v = tf.expand_dims(tf.map_fn(featurizer.vectorize, i), -1)
length_v = tf.cast(lengths * k, tf.int32)

In [8]:
v

<tf.Tensor 'ExpandDims:0' shape=(?, ?, 80, 1) dtype=float32>

In [9]:
config = malaya_speech.config.conformer_small_decoder_config
small_transducer = transducer.rnn.Model(small_model, vocabulary_size = 100, **config)

In [10]:
p = tf.placeholder(tf.int32, [None, None])

In [11]:
small_logits = small_transducer([v, p], training = True)
small_logits

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


<tf.Tensor 'transducer/transducer_joint/transducer_joint_vocab/BiasAdd:0' shape=(?, ?, ?, 100) dtype=float32>

In [12]:
greedy_decoded = small_transducer.greedy_decoder(v)

In [13]:
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

In [14]:
y, sr = malaya_speech.load('../speech/example-speaker/husein-zolkepli.wav', sr = 16000)
y1, sr = malaya_speech.load('../speech/example-speaker/shafiqah-idayu.wav', sr = 16000)
len(y), len(y1)

(90090, 56298)

In [15]:
padded, lens = malaya_speech.padding.sequence_1d([y, y1], return_len = True)
t = [[1,1,1,1,1,1], [1,1,1,1,1,1]]

In [16]:
sess.run(small_logits, feed_dict = {i: padded, lengths: lens, p: t}).shape

(2, 141, 6, 100)

In [17]:
%%time

greedy = sess.run(greedy_decoded, feed_dict = {i: padded})
greedy.shape

CPU times: user 2.71 s, sys: 258 ms, total: 2.97 s
Wall time: 2.03 s


(2, 561)

In [18]:
greedy[0]

array([ 0, 62, 62, 62, 62, 62, 62, 62, 62, 62, 62, 62, 62, 62, 62, 62, 62,
       62,  8, 62, 62,  8, 62,  8, 62, 62, 62, 62, 62, 62, 62, 62, 62, 62,
        8, 62, 62, 62, 62, 62,  8, 62, 62, 62, 62, 62, 62, 62, 62, 62, 62,
        8,  8,  8, 62, 62, 62, 62, 62, 62, 62, 62, 62, 62, 62, 62, 62, 62,
       62, 62, 62, 62, 62, 62, 62, 62, 62, 62, 62, 62, 62, 62, 62, 62, 62,
        8, 62, 62,  8,  8,  8, 62, 62, 62, 62, 62, 62, 62, 62, 62, 62, 62,
       62, 62,  8,  8, 62, 62, 62,  8, 62, 62, 62, 62, 62,  8,  8, 62, 62,
       62,  8, 62, 62, 62, 62, 62, 62, 62, 62, 62, 62, 62, 62, 62, 62,  8,
       62, 62,  8, 62, 62, 62,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0

In [1]:
import tensorflow as tf

In [2]:
res = tf.TensorArray(dtype=tf.int32, size=10, dynamic_size=False, infer_shape=False)
res = res.write(0, (1, 2))
res = res.write(1, (1, 2, 3))

In [3]:
r = res.read(1)
res = res.write(1, r)

In [4]:
sess = tf.InteractiveSession()

In [6]:
sess.run(res.read(1))

array([1, 2, 3], dtype=int32)

In [7]:
sess.run(res.read(1))

array([1, 2, 3], dtype=int32)